In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import shapely.geometry as geometry

import utils

In [ ]:
%time
df = utils.load_primary_data("../data/LocationHistory2018-12.json")

In [ ]:
gdf = utils.enhance_with_countries(df)

In [ ]:
gdf["point"] = gdf.geometry
gdf.loc[gdf.country != "Belgium", "point"] = np.NaN

In [ ]:
context_size = 5

for i in range(1, context_size):
    gdf["next-" + str(i)] = gdf.point.shift(i)
    
gdf.dropna(inplace=True)
gdf["geometry"] = gdf.apply(
    lambda row: geometry.LineString(
        [
            row["next-" + str(i)] 
            for i 
            in range(1, context_size)
        ]
    )
    , axis=1
)

gdf.drop(
    [
        "next-" + str(i)
        for i 
        in range(1, context_size)
    ],
    axis=1,
    inplace=True
)

In [ ]:
partial_gdf = gdf.sample(1)

x_max_with_border, x_min_with_border, y_max_with_border, y_min_with_border = (
    utils.determine_extrema_with_border(partial_gdf, absolute_border_size=0.003)
)

ax = utils.BELGIUM_ROADS.plot(edgecolor="gray", figsize=(10, 6), zorder=-1)
partial_gdf.plot(ax=ax, edgecolor="red", zorder=0)

plt.xlim([x_min_with_border, x_max_with_border])
plt.ylim([y_min_with_border, y_max_with_border])

partial_gdf.geometry